In [3]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import qrcode
from PIL import Image, ImageTk
import io
import customtkinter
# Global variable to store the last generated QR code image
current_qr_image = None

# Function to generate and display QR code
def generate_qrcode():
    global current_qr_image
    data = url_entry.get().strip()
    data_type = data_type_var.get()

    if not data:
        qr_display_label.config(text="Please enter valid data.", image='')
        current_qr_image = None
        return

    # Create the QR code with selected colors
    qr = qrcode.QRCode(version=2, box_size=5, border=2)
    qr.add_data(data)
    qr.make(fit=True)
    fill_color = fg_color_entry.get().strip() or 'black'
    back_color = bg_color_entry.get().strip() or 'white'
    img = qr.make_image(fill_color=fill_color, back_color=back_color)

    # Embed a logo if provided
    logo_path = logo_path_var.get()
    if logo_path:
        try:
            logo = Image.open(logo_path)
            logo = logo.resize((50, 50))  # Resize logo for embedding
            img = img.convert("RGBA")
            logo = logo.convert("RGBA")
            img.paste(logo, ((img.size[0] - logo.size[0]) // 2, (img.size[1] - logo.size[1]) // 2), mask=logo)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to embed logo: {e}")

    # Save the current QR code for potential download
    current_qr_image = img

    # Convert to a format displayable in Tkinter
    img_bytes = io.BytesIO()
    img.save(img_bytes, format='PNG')
    img_bytes.seek(0)
    tk_image = ImageTk.PhotoImage(Image.open(img_bytes))

    # Display the QR code in the label
    qr_display_label.config(image=tk_image)
    qr_display_label.image = tk_image
    qr_display_label.pack(pady=(10, 0))

# Function to download the generated QR code
def download_qrcode():
    if current_qr_image:
        file_path = filedialog.asksaveasfilename(defaultextension=".png",
                                                 filetypes=[("PNG files", "*.png"),
                                                            ("All files", "*.*")])
        if file_path:
            try:
                current_qr_image.save(file_path)
                messagebox.showinfo("Success", f"QR code saved to {file_path}")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to save the QR code: {e}")
    else:
        messagebox.showwarning("Warning", "Please generate a QR code before downloading.")

# Create the main window
root = tk.Tk()
root.title("Enhanced QR Code Generator")
root.geometry("450x600")
root.configure(bg="#f0f0f0")

# Header Frame
header_frame = ttk.Frame(root, padding=(10, 10))
header_frame.pack(fill="x", pady=(10, 5))

title_label = ttk.Label(header_frame, text="Enhanced QR Code Generator", font=("Helvetica", 16, "bold"))
title_label.pack()

# URL/Text Input Frame
input_frame = ttk.Frame(root, padding=(10, 10))
input_frame.pack(fill="x", pady=(10, 5))

data_type_var = tk.StringVar(value="URL")
url_radiobutton = ttk.Radiobutton(input_frame, text="URL", variable=data_type_var, value="URL")
url_radiobutton.grid(row=0, column=0, padx=5)
text_radiobutton = ttk.Radiobutton(input_frame, text="Text", variable=data_type_var, value="Text")
text_radiobutton.grid(row=0, column=1, padx=5)

url_label = ttk.Label(input_frame, text="Enter Data:", font=("Helvetica", 12))
url_label.grid(row=1, column=0, columnspan=2, sticky="w")

url_entry = ttk.Entry(input_frame, width=40, font=("Helvetica", 10))
url_entry.grid(row=2, column=0, columnspan=2, pady=(5, 10))

# Color Customization Inputs
fg_color_label = ttk.Label(input_frame, text="Foreground Color (e.g., black):")
fg_color_label.grid(row=3, column=0, sticky="w")
fg_color_entry = ttk.Entry(input_frame, width=20)
fg_color_entry.grid(row=3, column=1, pady=5)

bg_color_label = ttk.Label(input_frame, text="Background Color (e.g., white):")
bg_color_label.grid(row=4, column=0, sticky="w")
bg_color_entry = ttk.Entry(input_frame, width=20)
bg_color_entry.grid(row=4, column=1, pady=5)

# Logo Embedding Inputs
logo_label = ttk.Label(input_frame, text="Optional Logo Path:")
logo_label.grid(row=5, column=0, sticky="w")
logo_path_var = tk.StringVar()
logo_entry = ttk.Entry(input_frame, textvariable=logo_path_var, width=30)
logo_entry.grid(row=5, column=1, pady=5)

browse_logo_button = ttk.Button(input_frame, text="Browse", command=lambda: logo_path_var.set(filedialog.askopenfilename()))
browse_logo_button.grid(row=5, column=2, padx=5)

# Button to Generate QR Code
generate_button = ttk.Button(input_frame, text="Generate QR Code", command=generate_qrcode)
generate_button.grid(row=6, column=0, columnspan=3, pady=10)

# QR Code Display
qr_display_label = tk.Label(root, text="Your QR code will appear here", bg="#f0f0f0", font=("Helvetica", 10))
qr_display_label.pack(pady=5)

# Button to Download QR Code
download_button = ttk.Button(root, text="Download QR Code", command=download_qrcode)
download_button.pack(pady=10)

# Styling with ttk widgets
style = ttk.Style(root)
style.configure("TFrame", background="#f0f0f0")
style.configure("TLabel", background="#f0f0f0", font=("Helvetica", 10))
style.configure("TButton", font=("Helvetica", 10), padding=5)

# Run the Tkinter event loop
root.mainloop()
